In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pyCLIF
from datetime import timedelta
import pyarrow
import waterfall
import warnings
warnings.filterwarnings('ignore')

✅ Loaded configuration from config.json


In [2]:
clif_c2d2_mapping = pd.read_excel('../mapping/ccm-53-e1045-s002.xlsx')
cohort = pd.read_parquet('../output/final/c2d2_cohort.parquet')

## 🗺️ CLIF Vitals to C2D2 Mapping

This section outlines the standardized mapping of vital sign data from **CLIF source tables** to **C2D2-compliant variables** for downstream harmonization and modeling.


In [3]:
mapper = clif_c2d2_mapping[(clif_c2d2_mapping['Domain']=='Objective assessments') & (clif_c2d2_mapping['CLIF Table']=='Vitals')]
mapper.head()

,Domain,Sub-domain,Concept,Common Data Element,CLIF Table,CLIF mCIDE Crosswalk column,CLIF version,mCIDE,Definition,Coding,...,SOFA,PSOFA,nSOFA,PRISM III,PIM3,Charlson CI,MRC-ICU,SOI count,No SOI score,Potential EHR Datasource
205,Objective assessments,Vital signs,PRISM Heart Rate - highest,prism_heartrate_max,Vitals,vital_category,CLIF-2.1.0 (Live),heart_rate,Patient's heart rate (highest) captured during...,[integer],...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2,NaN,Flowsheet
206,Objective assessments,Vital signs,Heart Rate - highest,icu_24hr_heartrate_max,Vitals,vital_category,CLIF-2.1.0 (Live),heart_rate,Patient's heart rate (highest) captured during...,[integer],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Flowsheet
207,Objective assessments,Vital signs,MAP - highest,icu_24hr_map_max,Vitals,vital_category,CLIF-2.1.0 (Live),map,A-line or non-invasive (NIBP); both acceptable...,[integer] \nUnit: mmHg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Flowsheet
208,Objective assessments,Severity of Illness Score,PRISM Pupil reaction,prism_reactivepupils,Vitals,vital_category,CLIF 2.2 (Upcoming release),TBD,Number of pupils dilated >3 mm and fixed captu...,"None, One; Both; Unknown",...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1,NaN,Flowsheet
209,Objective assessments,Severity of Illness Score,Pupil reaction,icu_24hr_reactivepupils,Vitals,vital_category,CLIF 2.2 (Upcoming release),TBD,Number of pupils dilated >3 mm and fixed captu...,"None, One; Both; Unknown",...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1,NaN,Flowsheet


In [4]:
vitals_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'vital_category',
    'vital_value'
]

vitals_of_interest = ['heart_rate', 'map', 'temp_c']    #['respiratory_rate', 'sbp', 'dbp', 'spo2', 'weight_kg', 'height_cm']

In [5]:
vitals_cohort = pyCLIF.load_data('clif_vitals',
    columns=vitals_required_columns,
    filters={'hospitalization_id': cohort['hospitalization_id'].unique().tolist(), 
             'vital_category': vitals_of_interest}
)
vitals_cohort = pyCLIF.convert_datetime_columns_to_site_tz(vitals_cohort, pyCLIF.helper['timezone'])

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Data loaded successfully from C:/Users/vchaudha/Downloads/rush_parquet_2\clif_vitals.parquet


In [6]:
# Select relevant columns and drop duplicates
cohort_trimmed = cohort[['hospitalization_id', '24hr_icu_in_dttm', '24hr_icu_out_dttm']].drop_duplicates()

# Perform inner join on 'hospitalization_id'
vital_joined_df = pd.merge(cohort_trimmed, vitals_cohort, on='hospitalization_id', how='inner')

In [7]:
# Filter rows where recorded_dttm is within the ICU 24hr window
vital_filtered_df = vital_joined_df[
    (vital_joined_df['recorded_dttm'] >= vital_joined_df['24hr_icu_in_dttm']) &
    (vital_joined_df['recorded_dttm'] <= vital_joined_df['24hr_icu_out_dttm'])
]

# Group by hospitalization_id and vital_category, and compute min/max of vital_value
vital_agg_df = (
    vital_filtered_df
    .groupby(['hospitalization_id', 'vital_category'])['vital_value']
    .agg(['min', 'max'])
    .unstack(level='vital_category')  # Pivot vital_category into columns
)
# Rename columns to format: vital_category_min, vital_category_max
vital_agg_df.columns = [f'icu_24hr_{vital}_{stat}' for stat, vital in vital_agg_df.columns]

In [8]:
vital_agg_df.reset_index().to_parquet('../output/final/c2d2_objective_assessments.parquet',index=False)